In [1]:
import os
import cv2
import numpy as np
import xml.etree.cElementTree as ET

In [2]:
prototxt_path = r'../data/Create-Face-Data-from-Images-master/model_data/deploy.prototxt'
caffemodel_path = r'../data/Create-Face-Data-from-Images-master/model_data/weights.caffemodel'
base_dir = r'../'

In [3]:
model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

In [4]:
# Create directory 'updated_images' if it does not exist
raw_path = os.path.join(base_dir, r'data/raw1')
if not os.path.exists(raw_path):
    print("New directory created")
    os.makedirs(raw_path)

In [5]:
# Create directory 'faces' if it does not exist
faces_path = os.path.join(base_dir, r'data/faces')
if not os.path.exists(faces_path):
    print("New directory created")
    os.makedirs(faces_path)

In [6]:
# Create directory 'bbox' if it does not exist
bbox_path = os.path.join(base_dir, r'data/bbox')
if not os.path.exists(bbox_path):
    print("New directory created")
    os.makedirs(bbox_path)

In [47]:
root = ET.Element("annotation")
ET.SubElement(root, "folder").text = "raw"
source = ET.SubElement(root, "source")
ET.SubElement(source, "field1", name="blah", pro='prop1').text = "C:/"
tree = ET.ElementTree(root)
tree.write(os.path.join(bbox_path, "test.xml"))

In [33]:
# root = ET.Element("annotation")
# folder = ET.SubElement(root, "folder")
# folder.text = "raw"
# source = ET.SubElement(root, "source")
# ET.SubElement(source, "field1", name="blah", pro='prop1').text = "C:/"
# tree = ET.ElementTree(root)
# tree.write(os.path.join(bbox_path, "test.xml"))

def create_bbox_annotation(image_dict, detections):
#     image_dict:
#         filename - string, with extension
#         width - int
#         height - int
#         depth - int
    
    
    folder_t = 'raw'
    filename_t = image_dict['filename']
    path_t = r"D:\Documents\Study\jupyter_notebook\haircut\data\raw"
    database_t = "Unknown"
    segmented_t = 0
    name_t = "head"
    pose_t = "unspecified"
    truncate_t = "1"
    difficult_t = "0"
    
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = folder_t
    ET.SubElement(root, "filename").text = filename_t
    ET.SubElement(root, "path").text = path_t
    
    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = database_t
    
    ET.SubElement(root, "segmented").text = segmented_t
    s = ET.SubElement(root, "size")
    for i in ['width', 'height', 'depth']:
        ET.SubElement(s, i).text = str(image_dict[i])
    ET.SubElement(root, "segmented").text = segmented_t
    
    for i in range(0, detections.shape[2]):
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        confidence = detections[0, 0, i, 2]
        if confidence > 0.8:
            f_height = endY - startY
            offsetX = int((endX - startX)/2)
    #         frame = image[max(0, startY - int(f_height/(2.3))):(endY + int(f_height/4)), max(0, startX - offsetX):endX + offsetX]
            obj = ET.SubElement(root, "object")
            ET.SubElement(obj, "name").text = name_t
            ET.SubElement(obj, "pose").text = pose_t
            ET.SubElement(obj, "truncated").text = truncate_t
            ET.SubElement(obj, "difficult").text = difficult_t
            bbox = ET.SubElement(obj, "bndbox")
            ET.SubElement(bbox, "xmin").text = str(max(0, startX - offsetX))
            ET.SubElement(bbox, "ymin").text = str(max(0, startY - int(f_height/(2.3))))
            ET.SubElement(bbox, "xmax").text = str(endX + offsetX)
            ET.SubElement(bbox, "ymax").text = str((endY + int(f_height/4)))
    tree = ET.ElementTree(root)
    tree.write(os.path.join(bbox_path, image_dict['filename'].split('.')[0] + ".xml"))    

In [34]:
# Loop through all images and save images with marked faces
for file in os.listdir(raw_path)[:7]:
    file_name, file_extension = os.path.splitext(file)
    if (file_extension in ['.png','.jpg']):
        print("Image path: {}".format(raw_path + '/' + file))
    image = cv2.imread(os.path.join(raw_path, file))
    
    height = len(image)
    width = len(image[0])
    channel = len(image[0][0])
    
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    model.setInput(blob)
    detections = model.forward() 
    
    image_dict = {
        'filename': file,
        'height': height,
        'width': width,
        'depth': channel
    }
    create_bbox_annotation(image_dict, detections)
    
#     # Create frame around face
#     for i in range(0, detections.shape[2]):
#         box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
#         (startX, startY, endX, endY) = box.astype("int")
#         confidence = detections[0, 0, i, 2]
#         offsetY = int((endY - startY)/(2.2))
#         offsetX = int((endX - startX)/2)
#         # If confidence > 0.5, show box around face
#         if (confidence > 0.5):
#             pass
# #             cv2.rectangle(image, (startX, startY), (endX, endY), (255, 255, 255), 2)
# #             cv2.rectangle(image, (max(0,startX-offsetX), max(0,startY-offsetY)), (endX+offsetX, endY+offsetY), (255, 255, 255), 2)
            
# #     cv2.imwrite('updated_images/' + file, image)
# #     print("Image " + file + " converted successfully")
    
#     # Identify each face
#     count = 0
#     for i in range(0, detections.shape[2]):
#         box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
#         (startX, startY, endX, endY) = box.astype("int")
#         confidence = detections[0, 0, i, 2]
#         # If confidence > 0.5, save it as a separate file
#         if (confidence > 0.5):
#             count += 1
#             offsetY = (endY - startY)//2
#             offsetX = (endX - startX)//2
# #             frame = image[max(0, startY - offsetY):(endY + offsetY), max(0, startX-offsetX):endX+offsetX]
# #             frame = image[startY:endY, startX:endX]
#             frame = image[max(0, startY - int((endY - startY)//2.3)):(endY + (endY - startY)//4), max(0, startX-offsetX):endX+offsetX]

#             cv2.imwrite(os.path.join(faces_path, str(i) + '__' + file), frame)
    
    

Image path: ../data/raw1/01.jpg
Image path: ../data/raw1/02.jpg
Image path: ../data/raw1/03.jpg
Image path: ../data/raw1/04.jpg
Image path: ../data/raw1/05.jpg
Image path: ../data/raw1/06.jpg
Image path: ../data/raw1/07.jpg
